In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [3]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", [foo_data, foo_data, None], foo_input, foo_head)
bar = Task("bar_task", [bar_data, bar_data, None], bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo]

In [7]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=4, 
    lr=0.05, 
    progress_bar=True,
    checkpoint_metric="foo_task/valid/accuracy",
)

[1 epo]: TRAIN:[foo_task/loss=0.047] VALID:[foo_task/accuracy=0.992]
Saving model at iteration 1 with best score 0.992



[2 epo]: TRAIN:[foo_task/loss=0.037] VALID:[foo_task/accuracy=0.995]
Saving model at iteration 2 with best score 0.995



[3 epo]: TRAIN:[foo_task/loss=0.036] VALID:[foo_task/accuracy=0.995]



[4 epo]: TRAIN:[foo_task/loss=0.035] VALID:[foo_task/accuracy=0.997]
Saving model at iteration 4 with best score 0.997

Restoring best model from iteration 4 with score 0.997
Finished Training
{'foo_task/valid/accuracy': 0.997}


In [5]:
for batch in foo.data_loaders[0]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

{'foo_task': tensor([[ -2.8308,   3.0709],
        [ 13.5472, -13.5081],
        [ 11.0965, -11.1687],
        [ -4.4102,   4.2775]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor(0.0007, grad_fn=<NllLossBackward>)}
{'foo_task': tensor([[2.7273e-03, 9.9727e-01],
        [1.0000e+00, 1.7784e-12],
        [1.0000e+00, 2.1396e-10],
        [1.6863e-04, 9.9983e-01]])}
